In [1]:
import torch
from src.model import Conv_Net_Dropout, Conv_Net_FC_Dropout
from src.Datasets import load
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
model = Conv_Net_FC_Dropout()
model.load_state_dict(torch.load('/home/julius/OOD/models/model3_drop_fc_0_5.pth'))
model.to('cuda')
model.eval()

Conv_Net_FC_Dropout(
  (convs): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (7): ReLU()
  )
  (fcs): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=3136, out_features=128, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=128, out_features=10, bias=True)
    (5): Softmax(dim=1)
  )
)

In [3]:
mnist_val = load('MNIST')(train=False)
mnist_loader_val = DataLoader(mnist_val, batch_size=128, shuffle=False)

mnist_train = load('MNIST')(train=True)
mnist_loader_train = DataLoader(mnist_train, batch_size=128, shuffle=False)

In [4]:
prediction_val = []
with torch.no_grad():
    for x, y in tqdm(mnist_loader_val):
        x = x.to('cuda')
        y = y.to('cuda')
        out = model(x)
        prediction_val.append(out.argmax(dim=1).cpu() == y.cpu())
        

100%|██████████| 79/79 [00:00<00:00, 116.76it/s]


In [5]:
prediction_train = []
with torch.no_grad():
    for x, y in tqdm(mnist_loader_train):
        x = x.to('cuda')
        y = y.to('cuda')
        out = model(x)
        prediction_train.append(out.argmax(dim=1).cpu() == y.cpu())

100%|██████████| 469/469 [00:03<00:00, 151.11it/s]


In [6]:
print('Validation Accuracy: ', torch.cat(prediction_val).float().mean().item())
print('Train Accuracy: ', torch.cat(prediction_train).float().mean().item())

Validation Accuracy:  0.993399977684021
Train Accuracy:  0.9953500032424927
